In [1]:
import pandas as pd
import time

In [2]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])

In [3]:
member_info=member_info[['uid','l_topic','topic_n']]

ques_info['qtopic'] = ques_info['qtopic'].map(lambda x:x.split(','))
ques_info['qtopic_len'] = ques_info['qtopic'].map(lambda x:len(x))

member_info['l_topic'] = member_info['l_topic'].map(lambda x:x.split(','))
member_info['l_topic_len'] = member_info['l_topic'].map(lambda x:len(x))

In [4]:
from tqdm import tqdm
tqdm.pandas()
def flatten_topic(df):    
    u = []
    a = []
    q=[]
    for i in tqdm(range(len(df['qtopic'].values))):
        u += [df['qid'].values[i]]*df['qtopic_len'].values[i]
        a += list(df['qtopic'].values[i])
    new_df = pd.DataFrame()
    new_df['qid'] = u
    new_df['qtopic'] = a
    return new_df

In [5]:
q_df=flatten_topic(ques_info)

100%|██████████| 1829900/1829900 [00:34<00:00, 52870.20it/s]


In [6]:
def flatten_topic(df):    
    u = []
    a = []
    q=[]
    for i in tqdm(range(len(df['l_topic'].values))):
        u += [df['uid'].values[i]]*df['l_topic_len'].values[i]
        a += list(df['l_topic'].values[i])
    new_df = pd.DataFrame()
    new_df['uid'] = u
    new_df['l_topic'] = a
    return new_df

In [8]:
u_df=flatten_topic(member_info)

100%|██████████| 1931654/1931654 [00:37<00:00, 51662.17it/s]


In [10]:
q_df=q_df.groupby(['qtopic']).agg('count').reset_index()
u_df=u_df.groupby(['l_topic']).agg('count').reset_index()

In [12]:
q_df=q_df.set_index('qtopic').T.to_dict()
u_df=u_df.set_index('l_topic').T.to_dict()

In [16]:
ques_info=ques_info[['qid','qtopic','qtopic_len']]

In [21]:
def cal_(x):
    r=0
    for i in x:
        try:
            r+=u_df[i]['uid']
        except:
            pass
    return r

In [22]:
ques_info['topic_countuid_sum']=ques_info['qtopic'].progress_apply(lambda x:cal_(x))


100%|██████████| 1829900/1829900 [00:09<00:00, 194125.90it/s]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
def cal_(x):
    r=0
    for i in x:
        try:
            r+=q_df[i]['qid']
        except:
            pass
    return r

In [25]:
member_info['topic_countqid_sum']=member_info['l_topic'].progress_apply(lambda x:cal_(x))


100%|██████████| 1931654/1931654 [00:20<00:00, 95588.91it/s]


In [26]:
ques_info['topic_countqid_sum_q']=ques_info['qtopic'].progress_apply(lambda x:cal_(x))


100%|██████████| 1829900/1829900 [00:09<00:00, 193946.38it/s]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
def cal_(x):
    r=0
    for i in x:
        try:
            r+=u_df[i]['uid']
        except:
            pass
    return r

In [28]:
member_info['topic_countqid_sum_u']=member_info['l_topic'].progress_apply(lambda x:cal_(x))


100%|██████████| 1931654/1931654 [00:19<00:00, 99308.41it/s]


In [32]:
member_info['topic_countqid_mean']=member_info['topic_countqid_sum']/member_info['l_topic_len']
member_info['topic_countqid_mean_u']=member_info['topic_countqid_sum_u']/member_info['l_topic_len']
ques_info['topic_countuid_mean']=ques_info['topic_countuid_sum']/ques_info['qtopic_len']
ques_info['topic_countuid_mean_q']=ques_info['topic_countqid_sum_q']/ques_info['qtopic_len']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [36]:
ques_info[['qid','topic_countuid_mean','topic_countuid_sum','topic_countuid_mean_q','topic_countqid_sum_q']].to_csv('feature_qid_topic_uid_count.csv',index=False)
member_info[['uid','topic_countqid_mean','topic_countqid_mean_u','topic_countqid_sum','topic_countqid_sum_u']].to_csv('feature_uid_topic_qid_count.csv',index=False)